In [ ]:
# Libraries

# Standard library imports
import os
import logging

# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Variables

# Directory files should be downloaded to 
download_dir= os.path.abspath("./Downloads")

# Directory if using a custom chrome profile
profile_dir = r"C:\Users\13613100\AppData\Local\Google\Chrome\Developer"

# If the browser should be headless (not recommended)
headless = False

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler("sandbox.log")]
)

In [ ]:
# Create a Chrome WebDriver with Options
def create_driver(headless=False, profile_dir=None, download_dir=None):
    chrome_options = Options()

    # Optional: Run headless for environments without GUI
    if headless:
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920x1080")  # Set default window size for headless
    
    # Optional: Load custom profile (to retain session, cookies, etc.)
    if profile_dir:
        chrome_options.add_argument(f"user-data-dir={profile_dir}")
    
    # Stability options
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-cache")
    chrome_options.add_argument("--disable-extensions")

    # Download preferences
    if download_dir:
        # Create download directory if it doesn't exist
        os.makedirs(download_dir, exist_ok=True)
        logging.info(f"Download directory set to: {download_dir}")
        
        prefs = {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "plugins.always_open_pdf_externally": True
        }
        chrome_options.add_experimental_option("prefs", prefs)
    
    # Create and return the Chrome WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.implicitly_wait(10)  # Set default implicit wait
    logging.info("Chrome WebDriver created successfully.")
    return driver


In [ ]:
logging.info("Initializing WebDriver...")
driver = create_driver(
    headless = headless,
    profile_dir = profile_dir,
    download_dir = download_dir
    )
logging.info("WebDriver initialized.")